# Adding Sentiment of titles

In [4]:
# Import relevant libraries
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [5]:
# UTF-8 encoding setup and Vader Sentiment Analysis example
import locale
import sys
import os

# Set UTF-8 encoding
if sys.platform.startswith('win'):
    # For Windows
    os.environ['PYTHONIOENCODING'] = 'utf-8'
else:
    # For macOS/Linux
    locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

# Ensure pandas displays UTF-8 characters properly
pd.set_option('display.unicode.east_asian_width', True)
pd.set_option('display.unicode.ambiguous_as_wide', True)

In [3]:
# Import datasets

df0_cleaned = pd.read_parquet('data/cleaned_data_0.parquet')
df1_cleaned = pd.read_parquet('data/cleaned_data_1.parquet')
df2_cleaned = pd.read_parquet('data/cleaned_data_2.parquet')    
df3_cleaned = pd.read_parquet('data/cleaned_data_3.parquet')
df4_cleaned = pd.read_parquet('data/cleaned_data_4.parquet')

# check first rows for df0_cleaned
df0_cleaned.head()


,ID,publishedAt,instances,source-id,source-name,en-title,language,category,year,month
0,12436,2020-08-06T09:21:27Z,"[{'category': 'general', 'collectedAt': '2020-...",None,Albidda.net,A doctor warns of new symptoms of “Corona” tha...,ar,general,2020,8
1,12541,2020-08-06T15:45:39Z,"[{'category': 'general', 'collectedAt': '2020-...",None,Middle East Online,Foldable phones lead Samsung to climb the top ...,ar,general,2020,8
2,12568,2020-08-06T16:43:05Z,"[{'category': 'general', 'collectedAt': '2020-...",None,Alanba.com.kw,Explosive stars create calcium in - Kuwait New...,ar,general,2020,8
3,12795,2020-08-07T07:08:19Z,"[{'category': 'general', 'collectedAt': '2020-...",None,Al-ain.com,Twitter secures the accounts of governments an...,ar,general,2020,8
4,13129,2020-08-07T13:33:00Z,"[{'category': 'general', 'collectedAt': '2020-...",None,محليات,Corona patients without symptoms carry a viral...,ar,general,2020,8


In [6]:
# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Test with a sample text to ensure UTF-8 works
sample_text = "This is a great day! 😊"
print(f"Sample text: {sample_text}")
print(f"Sentiment scores: {analyzer.polarity_scores(sample_text)}")

Sample text: This is a great day! 😊
Sentiment scores: {'neg': 0.0, 'neu': 0.402, 'pos': 0.598, 'compound': 0.8858}


In [9]:
# Add sentiment scores to each dataframe based on the 'title' column

def add_sentiment_scores(df):
    # Create a copy to avoid modifying the original dataframe
    df_copy = df.copy()
    
    # Handle missing or null values in the 'en-title' column
    def safe_sentiment_analysis(text):
        if pd.isna(text) or text is None or text == '':
            # Return neutral sentiment for missing/empty text
            return {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
        try:
            return analyzer.polarity_scores(str(text))
        except Exception as e:
            print(f"Error processing text: {text[:50]}... Error: {e}")
            return {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
    
    sentiments = df_copy['en-title'].apply(safe_sentiment_analysis)
    sentiment_df = pd.DataFrame(list(sentiments))
    df_result = pd.concat([df_copy.reset_index(drop=True), sentiment_df.reset_index(drop=True)], axis=1)
    return df_result

df0_sentiment = add_sentiment_scores(df0_cleaned)
df1_sentiment = add_sentiment_scores(df1_cleaned)
df2_sentiment = add_sentiment_scores(df2_cleaned)
df3_sentiment = add_sentiment_scores(df3_cleaned)
df4_sentiment = add_sentiment_scores(df4_cleaned)

# check first rows for df0_sentiment
df0_sentiment.head()

,ID,publishedAt,instances,source-id,source-name,en-title,language,category,year,month,neg,neu,pos,compound
0,12436,2020-08-06T09:21:27Z,"[{'category': 'general', 'collectedAt': '2020-...",None,Albidda.net,A doctor warns of new symptoms of “Corona” tha...,ar,general,2020,8,0.149,0.851,0.000,-0.2023
1,12541,2020-08-06T15:45:39Z,"[{'category': 'general', 'collectedAt': '2020-...",None,Middle East Online,Foldable phones lead Samsung to climb the top ...,ar,general,2020,8,0.000,0.893,0.107,0.2023
2,12568,2020-08-06T16:43:05Z,"[{'category': 'general', 'collectedAt': '2020-...",None,Alanba.com.kw,Explosive stars create calcium in - Kuwait New...,ar,general,2020,8,0.000,0.769,0.231,0.2732
3,12795,2020-08-07T07:08:19Z,"[{'category': 'general', 'collectedAt': '2020-...",None,Al-ain.com,Twitter secures the accounts of governments an...,ar,general,2020,8,0.000,0.881,0.119,0.3182
4,13129,2020-08-07T13:33:00Z,"[{'category': 'general', 'collectedAt': '2020-...",None,محليات,Corona patients without symptoms carry a viral...,ar,general,2020,8,0.176,0.824,0.000,-0.4939


In [11]:
# Add sentiment scores based on compound score where:
# positive sentiment: compound score >= 0.05
# neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
# negative sentiment: compound score <= -0.05

def categorize_sentiment(compound_score):
    if compound_score >= 0.05:
        return 'positive'
    elif compound_score <= -0.05:
        return 'negative'
    else:
        return 'neutral'
    
df0_sentiment['sentiment_category'] = df0_sentiment['compound'].apply(categorize_sentiment)
df1_sentiment['sentiment_category'] = df1_sentiment['compound'].apply(categorize_sentiment)
df2_sentiment['sentiment_category'] = df2_sentiment['compound'].apply(categorize_sentiment)
df3_sentiment['sentiment_category'] = df3_sentiment['compound'].apply(categorize_sentiment)
df4_sentiment['sentiment_category'] = df4_sentiment['compound'].apply(categorize_sentiment)

# check first rows for df0_sentiment with sentiment category
df0_sentiment.head()

,ID,publishedAt,instances,source-id,source-name,en-title,language,category,year,month,neg,neu,pos,compound,sentiment_category
0,12436,2020-08-06T09:21:27Z,"[{'category': 'general', 'collectedAt': '2020-...",None,Albidda.net,A doctor warns of new symptoms of “Corona” tha...,ar,general,2020,8,0.149,0.851,0.000,-0.2023,negative
1,12541,2020-08-06T15:45:39Z,"[{'category': 'general', 'collectedAt': '2020-...",None,Middle East Online,Foldable phones lead Samsung to climb the top ...,ar,general,2020,8,0.000,0.893,0.107,0.2023,positive
2,12568,2020-08-06T16:43:05Z,"[{'category': 'general', 'collectedAt': '2020-...",None,Alanba.com.kw,Explosive stars create calcium in - Kuwait New...,ar,general,2020,8,0.000,0.769,0.231,0.2732,positive
3,12795,2020-08-07T07:08:19Z,"[{'category': 'general', 'collectedAt': '2020-...",None,Al-ain.com,Twitter secures the accounts of governments an...,ar,general,2020,8,0.000,0.881,0.119,0.3182,positive
4,13129,2020-08-07T13:33:00Z,"[{'category': 'general', 'collectedAt': '2020-...",None,محليات,Corona patients without symptoms carry a viral...,ar,general,2020,8,0.176,0.824,0.000,-0.4939,negative
